<a href="https://colab.research.google.com/github/JALVARADORUIZ/Magister_BigData_UB/blob/main/M2_AG1_GRUPO14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNDAMENTOS DE ESTADÍSTICA**
PADAWANS:
- PAULA VALLEJOS
- JOSE ARMANDO ALVARADO RUIZ

# **Introducción al cálculo de probabilidades**



**Entrega: M2_AG1_CORONAVIRUS**

# Descripción de la tarea

Tenéis un dataset (worlddometer_covid_ue.csv) con los datos a nivel europeo del Covid-19. En el momento del estudio, se detecta la siguiente situación:

Hay 6 países europeos con el mayor número de casos por Covid-19 que son Rusia, España, UK, Francia, Italia y Alemania con, aproximadamente, 250.000 casos.

Por lo que sabemos, se han adoptado algunas medidas similares de contención (confinamiento, mascarilla, geles, etc.) aunque no se han aplicado en las mismas fechas (por ejemplo, Italia comenzó 2 semanas antes que España y, en UK, postergaron el confinamiento en más de 15 días con respecto a España).

El número de habitantes por población es similar, quitando a Rusia que tiene el triple de personas que en España. Esto hace que descartemos a Rusia del análisis comparativo. En este punto, cabría la opción de plantearse añadir a Ucrania que tiene un número similar de habitantes que España.

Con los datos facilitados sobre el total de casos confirmados con Covid-19 (suponemos que con PCR) y sobre el total de fallecimientos acontecidos, te pedimos que halles:

- La probabilidad de sobrevivir en cualquiera de los seis países europeos seleccionados para el estudio.
- Si seleccionamos una de las personas fallecidas, ¿cuál es la probabilidad de que residiera en España? ¿Y en el resto de países?

Justifica y comenta los resultados obtenidos.

CONSIDERACIONES

Fuente de la información.

Se supondrá que los datos de fallecimiento se han contabilizado exactamente igual en los países seleccionados para el análisis.

Los datos son oficiales, con fecha 29-08-2020, pero esto no quiere decir que se puedan extraer conclusiones definitivas, mejor dicho, hay que considerar más variables de las que se tienen en cuenta en este análisis como, por ejemplo, la distribución de la edad en la población, la forma de considerar los fallecimientos por Covid-19 (si son con PCR o no todos los que figuran), las medidas adoptadas, etc.

Justifica adecuadamente tus respuestas.

# Carga y análisis de la Data

IMPORTACIÓN DE LIBRERÍAS

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
import seaborn as sns
import math
import pandas.util.testing as tm
%matplotlib inline

# Carga de Datos y visualización general

url = 'https://raw.githubusercontent.com/md-lorente/data/master/worlddometer_covid_ue.csv'
df = pd.read_csv(url, sep=";", decimal=",")
df = df.infer_objects()
df.head(10)

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
0,0,Europe,"3,543,710",10.493,207.347,149.0,"2,063,335","1,273,028",5.619,NaN,NaN,NaN,NaN,NaN
1,1,Russia,"990,326",4.980,17.093,68.0,"806,982","166,251",2.300,6.786,117.0,"36,400,000","249,409","145,944,843"
2,2,Spain,"455,621",NaN,29.011,NaN,NaN,NaN,751.000,9.744,620.0,"8,517,446","182,161","46,757,783"
3,3,UK,"332,752",NaN,41.498,NaN,NaN,NaN,60.000,4.897,611.0,"16,273,209","239,508","67,944,393"
4,4,France,"272,530",NaN,30.602,NaN,"86,177","155,751",387.000,4.174,469.0,"6,175,223","94,571","65,297,356"
5,5,Italy,"266,853",NaN,35.473,NaN,"208,224","23,156",79.000,4.415,587.0,"8,509,618","140,778","60,447,003"
6,6,Germany,"242,825",NaN,9.363,NaN,"217,484","15,978",245.000,2.897,112.0,"11,208,091","133,703","83,828,051"
7,7,Ukraine,"119,074",2.096,2.527,35.0,"56,734","59,813",177.000,2.725,58.0,"1,536,171","35,161","43,689,434"
8,8,Romania,"86,785",952.000,3.578,39.0,"37,665","45,542",492.000,4.516,186.0,"1,795,633","93,445","19,216,003"
9,9,Belgium,"84,599",647.000,9.891,5.0,"18,403","56,305",72.000,7.294,853.0,"2,261,043","194,953","11,597,901"


In [80]:
df.columns

Index(['Num', 'Country', 'Total Cases', 'New Cases', 'Total Deaths',
       'New Deaths', 'Total Recovered', 'Active Cases', 'Serious Critical',
       'Total Cases per 1M pop', 'Deaths per 1M pop', 'Total Tests',
       'Tests per 1M pop', 'Population '],
      dtype='object')

In [81]:
df.dtypes

Num                         int64
Country                    object
Total Cases                object
New Cases                 float64
Total Deaths              float64
New Deaths                float64
Total Recovered            object
Active Cases               object
Serious Critical          float64
Total Cases per 1M pop    float64
Deaths per 1M pop         float64
Total Tests                object
Tests per 1M pop           object
Population                 object
dtype: object

In [82]:
# Transformamos las columnas del tipo object que representan numeros a float para un mejor analisis

df['Total Cases'] = df['Total Cases'].str.replace(',', '').astype(float)
df['Total Recovered'] = df['Total Recovered'].str.replace(',', '').astype(float)
df['Active Cases'] = df['Active Cases'].str.replace(',', '').astype(float)
df['Total Tests'] = df['Total Tests'].str.replace(',', '').astype(float)
df['Tests per 1M pop'] = df['Tests per 1M pop'].str.replace(',', '').astype(float)
df['Population '] = df['Population '].str.replace(',', '').astype(float)

In [83]:
df.head(10)

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
0,0,Europe,3543710.0,10.493,207.347,149.0,2063335.0,1273028.0,5.619,NaN,NaN,NaN,NaN,NaN
1,1,Russia,990326.0,4.980,17.093,68.0,806982.0,166251.0,2.300,6.786,117.0,36400000.0,249409.0,145944843.0
2,2,Spain,455621.0,NaN,29.011,NaN,NaN,NaN,751.000,9.744,620.0,8517446.0,182161.0,46757783.0
3,3,UK,332752.0,NaN,41.498,NaN,NaN,NaN,60.000,4.897,611.0,16273209.0,239508.0,67944393.0
4,4,France,272530.0,NaN,30.602,NaN,86177.0,155751.0,387.000,4.174,469.0,6175223.0,94571.0,65297356.0
5,5,Italy,266853.0,NaN,35.473,NaN,208224.0,23156.0,79.000,4.415,587.0,8509618.0,140778.0,60447003.0
6,6,Germany,242825.0,NaN,9.363,NaN,217484.0,15978.0,245.000,2.897,112.0,11208091.0,133703.0,83828051.0
7,7,Ukraine,119074.0,2.096,2.527,35.0,56734.0,59813.0,177.000,2.725,58.0,1536171.0,35161.0,43689434.0
8,8,Romania,86785.0,952.000,3.578,39.0,37665.0,45542.0,492.000,4.516,186.0,1795633.0,93445.0,19216003.0
9,9,Belgium,84599.0,647.000,9.891,5.0,18403.0,56305.0,72.000,7.294,853.0,2261043.0,194953.0,11597901.0


# Seleccion Universo

In [84]:
# Seleccionamos a los 6 paises que serán nuestro Universo

df_universo = df.loc[2:7]

In [85]:
df_universo

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
2,2,Spain,455621.0,NaN,29.011,NaN,NaN,NaN,751.0,9.744,620.0,8517446.0,182161.0,46757783.0
3,3,UK,332752.0,NaN,41.498,NaN,NaN,NaN,60.0,4.897,611.0,16273209.0,239508.0,67944393.0
4,4,France,272530.0,NaN,30.602,NaN,86177.0,155751.0,387.0,4.174,469.0,6175223.0,94571.0,65297356.0
5,5,Italy,266853.0,NaN,35.473,NaN,208224.0,23156.0,79.0,4.415,587.0,8509618.0,140778.0,60447003.0
6,6,Germany,242825.0,NaN,9.363,NaN,217484.0,15978.0,245.0,2.897,112.0,11208091.0,133703.0,83828051.0
7,7,Ukraine,119074.0,2.096,2.527,35.0,56734.0,59813.0,177.0,2.725,58.0,1536171.0,35161.0,43689434.0


Tasa de supervivencia.

El porcentaje de personas que estarán con vida en un determinado momento después del diagnóstico.